In [1]:
import os
import warnings
import random
import pandas as pd
from itertools import combinations

import networkx as nx
import spacy

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

random.seed(100)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps, copy_X=True, 

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
def generateNetwork(sentence_list, ent1_word_idx, ent2_word_idx):
    try:
        sentence = ' '.join(sentence_list)
        doc = nlp(sentence)
        edges = []
        for word_idx, token in enumerate(doc):
            for child in token.children:
                edges.append((token.i,
                          child.i))

        graph = nx.Graph(edges)
        ## If shortest path not found
        try:
            shortest_path_length = nx.shortest_path_length(graph, source=ent1_word_idx, target=ent2_word_idx)
            shortest_path = nx.shortest_path(graph, source=ent1_word_idx, target=ent2_word_idx)
        except: 
            shortest_path_length = "no_path_found"
            shortest_path = "no_path_found"
        return shortest_path_length, shortest_path
    except:
        print("NETWORK - NO PATH FOUND: ", sentence_list, ent1_word_idx, ent2_word_idx)
        return "no_path_found", "no_path_found"

In [4]:
def getIndex(row, sentence, entity):
    try:
        return sentence.index(entity.split(" ")[0]) + 1
    except:
        print("INDEX NOT FOUND - ", sentence, entity, entity.split(" ")[0])
        return -1

In [28]:
def getFeatures(file):
#     tagged_tokens_header = ["text", "pos", "dep", "head", "begin", "end", "word_index", "sent_index", "entity_id", "ent_type"]
    df = pd.read_table(file)
    filtered_df = df.groupby(["sent_index", "entity mention ID"], as_index=False).agg({'text': ' '.join, 'entity type': '-'.join, 'pos' : '-'.join, 'tag' : '-'.join, 'dep': '-'.join})
    ## Calculating entity-entity distances per sentence
#     sent_df = df.where(~df["entity type"].isna()).groupby(["sent_index"], as_index = False).agg({'text': list, 'entity type': list, 'pos' : list, 'tag' : list, 'dep': list, 'entity mention ID': list})
    sent_df = filtered_df.groupby(["sent_index"], as_index=False).agg({'text': list, 'entity type': list, 'pos' : list, 'tag' : list, 'dep': list})
    display(sent_df)
    text_pairs = []
    ent_type_pairs = []
    pos_pairs = []
    tag_pairs = []
    dep_pairs = []
    for index, row in sent_df.iterrows():
        text_pairs.append(list(combinations(row["text"], r=2)))
        ent_type_pairs.append(list(combinations(row["entity type"], r=2)))
        pos_pairs.append(list(combinations(row["pos"], r=2)))
        dep_pairs.append(list(combinations(row["dep"], r=2)))
        tag_pairs.append(list(combinations(row["tag"], r=2)))
        
    sent_entity_df = pd.DataFrame()
    sent_entity_df["sent_index"] = sent_df["sent_index"]
    sent_entity_df["entity_pairs"] = text_pairs
    sent_entity_df["ent_type_pairs"] = ent_type_pairs
    sent_entity_df["pos_pairs"] = pos_pairs
    sent_entity_df["dep_pairs"] = dep_pairs
    sent_entity_df["tag_pairs"] = tag_pairs


    final_df = sent_entity_df.set_index('sent_index').apply(lambda x: x.apply(pd.Series).stack()).reset_index().drop('level_1', 1)
    
    
    sentences = pd.DataFrame()
    text = []
    root_word = ""
    root_word_index = -1
    curr_sent = 0
    for index, row in df.iterrows():
        if str(row["text"]).strip() == ".":
            text.append(row["text"])
            df_temp = {"sent_index": curr_sent, "sentence": text, "sentence_length": len(text), "root_word": root_word, "root_index": root_word_index}
            sentences = sentences.append(df_temp, ignore_index=True)
            text = []
            root_word = ""
            root_word_index = -1
            curr_sent = curr_sent + 1
        elif str(row["dep"]).strip() == "ROOT":
            root_word = row["text"]
            root_word_index = len(text) + 1
            text.append(row["text"])
        else:
            text.append(row["text"])

    display(sentences)
    final_df = final_df.join(sentences, on="sent_index", lsuffix="final", rsuffix="root")
    final_df = final_df.rename({"sent_indexfinal": "sent_index"}, axis='columns')
    
    display(final_df)
    final_df["entity_1_index"] = final_df.apply(lambda x : getIndex(x, x['sentence'], x['entity_pairs'][0]), axis=1)
    final_df["entity_2_index"] = final_df.apply(lambda x : getIndex(x, x['sentence'], x['entity_pairs'][1]), axis=1)
    final_df["entity_distance"] = final_df.apply(lambda x : x['entity_2_index'] - x['entity_1_index'], axis=1)
    final_df["no_words_before_entity_1"] = final_df["entity_1_index"] - 1 
    final_df["no_words_after_entity_2"] = final_df["sentence_length"] - final_df["entity_2_index"] - 1
    ## Calculating root-entity distances for each sentence
    final_df["entity_1_root_distance"] = final_df.apply(lambda x: x['root_index'] - x['entity_1_index'], axis=1)
    final_df["entity_2_root_distance"] = final_df.apply(lambda x: x['entity_2_index'] - x['root_index'], axis=1)
    # Expand tuples
    final_df[['entity 1 name', 'entity 2 name']] = final_df['entity_pairs'].apply(pd.Series)
    
    final_df[['entity_type_1', 'entity_type_2']] = final_df['ent_type_pairs'].apply(pd.Series)
    final_df[['entity_pos_1', 'entity_pos_2']] = final_df['pos_pairs'].apply(pd.Series)
    final_df[['entity_dep_1', 'entity_dep_2']] = final_df['dep_pairs'].apply(pd.Series)
    final_df[['entity_tag_1', 'entity_tag_2']] = final_df['tag_pairs'].apply(pd.Series)
    final_df[["shortest_distance", "shortest_path"]] = final_df.apply(lambda x : generateNetwork(x["sentence"], x["entity_1_index"], x["entity_2_index"]), axis = 1, result_type="expand")
        
    final_df = final_df.drop(["sent_indexroot", "entity_pairs", "ent_type_pairs", "pos_pairs", "dep_pairs", "sentence", "tag_pairs"], axis = 1)
    return final_df

In [29]:
columns = ['sent_index', 'sentence_length', 'root_word', 'root_index',
       'entity_1_index', 'entity_2_index', 'entity_distance',
       'no_words_before_entity_1', 'no_words_after_entity_2',
       'entity_1_root_distance', 'entity_2_root_distance', 'entity 1 name',
       'entity 2 name', 'entity_type_1', 'entity_type_2', 'entity_pos_1',
       'entity_pos_2', 'entity_dep_1', 'entity_dep_2', 'entity_tag_1',
       'entity_tag_2', 'shortest_distance', 'label']

def getFeaturesWithlabels(relation_file, tagged_tokens_file):
    final_df = getFeatures(tagged_tokens_file)
    relations = pd.read_table(relation_file)
    result = pd.merge(final_df, relations,  how='left', left_on=["entity 1 name", "entity 2 name"], right_on = ["entity 1 name", "entity 2 name"])
    result["label"] = result["rel type"].apply(lambda x : 1 if isinstance(x, str) else 0)
    return result[columns]
    

In [30]:
## For testing
tagged_tokens_file = "/Users/anishajauhari/Desktop/Independent Study /Dataset/relex/re3d/tagged_tokens/F7893F2F4E38C8E00C7D22E22E100428.csv"
relation_file = "/Users/anishajauhari/Desktop/Independent Study /Dataset/relex/re3d/ground_truth/F7893F2F4E38C8E00C7D22E22E100428.csv"
features_with_labels = getFeaturesWithlabels(relation_file, tagged_tokens_file)
display(features_with_labels)

## Entire Dataset
# ground_truth_path = "/Users/anishajauhari/Desktop/Independent Study /Dataset/relex/re3d/ground_truth"
# tagged_token_path = "/Users/anishajauhari/Desktop/Independent Study /Dataset/relex/re3d/tagged_tokens"

# os.chdir(tagged_token_path)
# features_with_labels = pd.DataFrame()
# for file in os.listdir():
#     if file.endswith(".csv"):
#         print("File : ", file)
#         tagged_tokens_file = f"{tagged_token_path}/{file}"
#         relation_file = f"{ground_truth_path}/{file}"
#         temp = getFeaturesWithlabels(relation_file, tagged_tokens_file)
#         features_with_labels = pd.concat([features_with_labels, temp])

# display(features_with_labels)

,sent_index,text,entity type,pos,tag,dep
0,0,"[U.S. Army General Joseph Votel, CENTCOM, U.S. Central Command, U.S. Air Force civilian employees]","[Person-Person-Person-Person-Person, Organisation, Organisation-Organisation-Organisation, Organisation-Organisation-Organisation-Organisation-Organisation]","[PROPN-PROPN-PROPN-PROPN-PROPN, PROPN, PROPN-PROPN-PROPN, PROPN-PROPN-PROPN-ADJ-NOUN]","[NNP-NNP-NNP-NNP-NNP, NNP, NNP-NNP-NNP, NNP-NNP-NNP-JJ-NNS]","[compound-compound-compound-compound-nsubj, compound, compound-compound-appos, compound-compound-nmod-amod-pobj]"
1,1,"[Votel, U.S. Central Command]","[Person, Organisation-Organisation-Organisation]","[PROPN, PROPN-PROPN-PROPN]","[NNP, NNP-NNP-NNP]","[nsubj, compound-compound-dobj]"
2,3,[allies],[Organisation],[NOUN],[NNS],[pobj]
3,6,[Votel],[Person],[PROPN],[NNP],[nsubj]
4,7,"[He, the command]","[Person, Organisation-Organisation]","[PRON, DET-NOUN]","[PRP, DT-NN]","[nsubj, det-pobj]"
5,8,"[The command, the civilian workforce, next year]","[Organisation-Organisation, Organisation-Organisation-Organisation, Temporal-Temporal]","[DET-NOUN, DET-ADJ-NOUN, ADJ-NOUN]","[DT-NN, DT-JJ-NN, JJ-NN]","[det-nsubj, det-amod-pobj, amod-npadvmod]"


,sent_index,sentence,sentence_length,root_word,root_index
0,0.0,"[U.S., Army, General, Joseph, Votel, ,, commander, ,, U.S., Central, Command, recently, met, with, U.S., Air, Force, civilian, employees, and, provided, an, overview, of, the, CENTCOM, vision, ,, the, current, mission, ,, and, strategic, approach, .]",36.0,met,13.0
1,1.0,"[\n \n, Votel, explained, U.S., Central, Command, ’s, top, priorities, :, \n\n, -, Ensure, an, effective, posture, .]",17.0,Ensure,13.0
2,2.0,"[\n\n, -, Strengthen, relationships, with, allies, .]",7.0,Strengthen,3.0
3,3.0,"[\n\n, -, Deter, and, counter, state, aggression, .]",8.0,aggression,7.0
4,4.0,"[\n\n, -, Disrupt, and, counter, violent, extremism, .]",8.0,extremism,7.0
5,5.0,"[\n\n, Votel, expressed, his, desire, that, the, command, attracts, and, retains, the, very, best, people, to, support, these, critical, priorities, and, that, they, are, fully, enabled, to, develop, and, contribute, to, the, mission, .]",34.0,expressed,3.0
6,6.0,"[\n\n, He, further, stressed, the, need, to, develop, our, future, leaders, through, timely, feedback, from, supervisors, ,, mentorship, programs, ,, and, opportunities, and, resources, for, growth, ,, while, noting, the, importance, of, timely, and, appropriate, recognition, for, superior, performance, throughout, an, employee, ’s, tenure, with, the, command, .]",48.0,stressed,4.0
7,7.0,"[\n\n, The, command, intends, to, hold, three, town, halls, next, year, with, varying, agendas, to, convey, useful, information, to, the, civilian, workforce, and, to, take, questions, and, recommendations, .]",29.0,intends,4.0


,sent_index,entity_pairs,ent_type_pairs,pos_pairs,dep_pairs,tag_pairs,sent_indexroot,sentence,sentence_length,root_word,root_index
0,0,"(U.S. Army General Joseph Votel, CENTCOM)","(Person-Person-Person-Person-Person, Organisation)","(PROPN-PROPN-PROPN-PROPN-PROPN, PROPN)","(compound-compound-compound-compound-nsubj, compound)","(NNP-NNP-NNP-NNP-NNP, NNP)",0.0,"[U.S., Army, General, Joseph, Votel, ,, commander, ,, U.S., Central, Command, recently, met, with, U.S., Air, Force, civilian, employees, and, provided, an, overview, of, the, CENTCOM, vision, ,, the, current, mission, ,, and, strategic, approach, .]",36.0,met,13.0
1,0,"(U.S. Army General Joseph Votel, U.S. Central Command)","(Person-Person-Person-Person-Person, Organisation-Organisation-Organisation)","(PROPN-PROPN-PROPN-PROPN-PROPN, PROPN-PROPN-PROPN)","(compound-compound-compound-compound-nsubj, compound-compound-appos)","(NNP-NNP-NNP-NNP-NNP, NNP-NNP-NNP)",0.0,"[U.S., Army, General, Joseph, Votel, ,, commander, ,, U.S., Central, Command, recently, met, with, U.S., Air, Force, civilian, employees, and, provided, an, overview, of, the, CENTCOM, vision, ,, the, current, mission, ,, and, strategic, approach, .]",36.0,met,13.0
2,0,"(U.S. Army General Joseph Votel, U.S. Air Force civilian employees)","(Person-Person-Person-Person-Person, Organisation-Organisation-Organisation-Organisation-Organisation)","(PROPN-PROPN-PROPN-PROPN-PROPN, PROPN-PROPN-PROPN-ADJ-NOUN)","(compound-compound-compound-compound-nsubj, compound-compound-nmod-amod-pobj)","(NNP-NNP-NNP-NNP-NNP, NNP-NNP-NNP-JJ-NNS)",0.0,"[U.S., Army, General, Joseph, Votel, ,, commander, ,, U.S., Central, Command, recently, met, with, U.S., Air, Force, civilian, employees, and, provided, an, overview, of, the, CENTCOM, vision, ,, the, current, mission, ,, and, strategic, approach, .]",36.0,met,13.0
3,0,"(CENTCOM, U.S. Central Command)","(Organisation, Organisation-Organisation-Organisation)","(PROPN, PROPN-PROPN-PROPN)","(compound, compound-compound-appos)","(NNP, NNP-NNP-NNP)",0.0,"[U.S., Army, General, Joseph, Votel, ,, commander, ,, U.S., Central, Command, recently, met, with, U.S., Air, Force, civilian, employees, and, provided, an, overview, of, the, CENTCOM, vision, ,, the, current, mission, ,, and, strategic, approach, .]",36.0,met,13.0
4,0,"(CENTCOM, U.S. Air Force civilian employees)","(Organisation, Organisation-Organisation-Organisation-Organisation-Organisation)","(PROPN, PROPN-PROPN-PROPN-ADJ-NOUN)","(compound, compound-compound-nmod-amod-pobj)","(NNP, NNP-NNP-NNP-JJ-NNS)",0.0,"[U.S., Army, General, Joseph, Votel, ,, commander, ,, U.S., Central, Command, recently, met, with, U.S., Air, Force, civilian, employees, and, provided, an, overview, of, the, CENTCOM, vision, ,, the, current, mission, ,, and, strategic, approach, .]",36.0,met,13.0
...,...,...,...,...,...,...,...,...,...,...,...
6,1,"(Votel, U.S. Central Command)","(Person, Organisation-Organisation-Organisation)","(PROPN, PROPN-PROPN-PROPN)","(nsubj, compound-compound-dobj)","(NNP, NNP-NNP-NNP)",1.0,"[\n \n, Votel, explained, U.S., Central, Command, ’s, top, priorities, :, \n\n, -, Ensure, an, effective, posture, .]",17.0,Ensure,13.0
7,7,"(He, the command)","(Person, Organisation-Organisation)","(PRON, DET-NOUN)","(nsubj, det-pobj)","(PRP, DT-NN)",7.0,"[\n\n, The, command, intends, to, hold, three, town, halls, next, year, with, varying, agendas, to, convey, useful, information, to, the, civilian, workforce, and, to, take, questions, and, recommendations, .]",29.0,intends,4.0
8,8,"(The command, the civilian workforce)","(Organisation-Organisation, Organisation-Organisation-Organisation)","(DET-NOUN, DET-ADJ-NOUN)","(det-nsubj, det-amod-pobj)","(DT-NN, DT-JJ-NN)",NaN,NaN,NaN,NaN,NaN
9,8,"(The command, next year)","(Organisation-Organisation, Temporal-Temporal)","(DET-NOUN, ADJ-NOUN)","(det-nsubj, amod-npadvmod)","(DT-NN, JJ-NN)",NaN,NaN,NaN,NaN,NaN


INDEX NOT FOUND -  ['\n\n', 'The', 'command', 'intends', 'to', 'hold', 'three', 'town', 'halls', 'next', 'year', 'with', 'varying', 'agendas', 'to', 'convey', 'useful', 'information', 'to', 'the', 'civilian', 'workforce', 'and', 'to', 'take', 'questions', 'and', 'recommendations', '.'] He He
INDEX NOT FOUND -  nan The command The
INDEX NOT FOUND -  nan The command The
INDEX NOT FOUND -  nan the civilian workforce the
INDEX NOT FOUND -  nan the civilian workforce the
INDEX NOT FOUND -  nan next year next
INDEX NOT FOUND -  nan next year next
NETWORK - NO PATH FOUND:  nan -1 -1
NETWORK - NO PATH FOUND:  nan -1 -1
NETWORK - NO PATH FOUND:  nan -1 -1


,sent_index,sentence_length,root_word,root_index,entity_1_index,entity_2_index,entity_distance,no_words_before_entity_1,no_words_after_entity_2,entity_1_root_distance,entity_2_root_distance,entity 1 name,entity 2 name,entity_type_1,entity_type_2,entity_pos_1,entity_pos_2,entity_dep_1,entity_dep_2,entity_tag_1,entity_tag_2,shortest_distance,label
0,0,36.0,met,13.0,1,26,25,0,9.0,12.0,13.0,U.S. Army General Joseph Votel,CENTCOM,Person-Person-Person-Person-Person,Organisation,PROPN-PROPN-PROPN-PROPN-PROPN,PROPN,compound-compound-compound-compound-nsubj,compound,NNP-NNP-NNP-NNP-NNP,NNP,7,0
1,0,36.0,met,13.0,1,1,0,0,34.0,12.0,-12.0,U.S. Army General Joseph Votel,U.S. Central Command,Person-Person-Person-Person-Person,Organisation-Organisation-Organisation,PROPN-PROPN-PROPN-PROPN-PROPN,PROPN-PROPN-PROPN,compound-compound-compound-compound-nsubj,compound-compound-appos,NNP-NNP-NNP-NNP-NNP,NNP-NNP-NNP,0,1
2,0,36.0,met,13.0,1,1,0,0,34.0,12.0,-12.0,U.S. Army General Joseph Votel,U.S. Air Force civilian employees,Person-Person-Person-Person-Person,Organisation-Organisation-Organisation-Organisation-Organisation,PROPN-PROPN-PROPN-PROPN-PROPN,PROPN-PROPN-PROPN-ADJ-NOUN,compound-compound-compound-compound-nsubj,compound-compound-nmod-amod-pobj,NNP-NNP-NNP-NNP-NNP,NNP-NNP-NNP-JJ-NNS,0,1
3,0,36.0,met,13.0,26,1,-25,25,34.0,-13.0,-12.0,CENTCOM,U.S. Central Command,Organisation,Organisation-Organisation-Organisation,PROPN,PROPN-PROPN-PROPN,compound,compound-compound-appos,NNP,NNP-NNP-NNP,7,0
4,0,36.0,met,13.0,26,1,-25,25,34.0,-13.0,-12.0,CENTCOM,U.S. Air Force civilian employees,Organisation,Organisation-Organisation-Organisation-Organisation-Organisation,PROPN,PROPN-PROPN-PROPN-ADJ-NOUN,compound,compound-compound-nmod-amod-pobj,NNP,NNP-NNP-NNP-JJ-NNS,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,1,17.0,Ensure,13.0,2,4,2,1,12.0,11.0,-9.0,Votel,U.S. Central Command,Person,Organisation-Organisation-Organisation,PROPN,PROPN-PROPN-PROPN,nsubj,compound-compound-dobj,NNP,NNP-NNP-NNP,3,0
7,7,29.0,intends,4.0,-1,20,21,-2,8.0,5.0,16.0,He,the command,Person,Organisation-Organisation,PRON,DET-NOUN,nsubj,det-pobj,PRP,DT-NN,no_path_found,0
8,8,NaN,NaN,NaN,-1,-1,0,-2,NaN,NaN,NaN,The command,the civilian workforce,Organisation-Organisation,Organisation-Organisation-Organisation,DET-NOUN,DET-ADJ-NOUN,det-nsubj,det-amod-pobj,DT-NN,DT-JJ-NN,no_path_found,0
9,8,NaN,NaN,NaN,-1,-1,0,-2,NaN,NaN,NaN,The command,next year,Organisation-Organisation,Temporal-Temporal,DET-NOUN,ADJ-NOUN,det-nsubj,amod-npadvmod,DT-NN,JJ-NN,no_path_found,0


In [21]:
def logistic_regression(x_train, x_test, y_train, y_test):
    print("--------Logistic Regression-----------")
    lr =  LogisticRegression(n_jobs=4)
    lr.fit(x_train, y_train)
    y_pred=lr.predict(x_test)
#     print("Coefficients: ", lr.coef_)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    print("Precision: ", metrics.precision_score(y_test, y_pred, average="binary", pos_label=1))
    print("Recall: ", metrics.recall_score(y_test, y_pred, average="binary", pos_label=1))
    print("F-Score: ", metrics.f1_score(y_test, y_pred, average="binary", pos_label=1))
    print(classification_report(y_test, y_pred))


In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
def random_forest(x_train, x_test, y_train, y_test):
    print("--------Random Forest-----------")
    clf = RandomForestClassifier(n_estimators=10, class_weight='balanced', n_jobs=4)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
#     print("Features: ", columns[:-1])
#     print("Coefficients: ", clf.feature_importances_)

    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    print("Precision: ", metrics.precision_score(y_test, y_pred, average="binary", pos_label=1))
    print("Recall: ", metrics.recall_score(y_test, y_pred, average="binary", pos_label=1))
    print("F-Score: ", metrics.f1_score(y_test, y_pred, average="binary", pos_label=1))
    print(classification_report(y_test, y_pred))

In [23]:
from sklearn.svm import SVC

def svm(kernel, x_train, x_test, y_train, y_test):
    print("--------Support Vector Classifier-----------")
    
    svm =  SVC(kernel=kernel, C=0.1, class_weight='balanced')
    svm.fit(x_train, y_train)
    y_pred = svm.predict(x_test)
#     if kernel == "linear":
#         print("Features: ", columns[:-1])
#         print("Coefficients: ", svm.coef_)

    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    print("Precision: ", metrics.precision_score(y_test, y_pred, average="binary", pos_label=1))
    print("Recall: ", metrics.recall_score(y_test, y_pred, average="binary", pos_label=1))
    print("F-Score: ", metrics.f1_score(y_test, y_pred, average="binary", pos_label=1))
    print(classification_report(y_test, y_pred))

In [24]:
## Convert to dummies
# define one hot encoding
X = features_with_labels.iloc[:,:-1]
y =  features_with_labels.iloc[:,-1]
onehot_encoder = OneHotEncoder(sparse=False)
X = onehot_encoder.fit_transform(X)

x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y)



ValueError: Input contains NaN

In [ ]:
logistic_regression(x_train, x_test, y_train, y_test)

In [ ]:
random_forest(x_train, x_test, y_train, y_test)

In [ ]:
svm("linear", x_train, x_test, y_train, y_test)

In [ ]:
svm("rbf", x_train, x_test, y_train, y_test)

In [ ]:
svm("poly", x_train, x_test, y_train, y_test)